<a href="https://colab.research.google.com/github/Jondoloh/NLP_Week-1_Lab-1-Naive-Bayes-Logistic_reg-/blob/main/John_Ndolo_NLP_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
train1 = '/content/drive/MyDrive/Colab Notebooks/NLP/Week 1/Lab 1/train1.txt'
#f=open(train1)
#f.read()
#f.close()

In [21]:
valid1 = '/content/drive/MyDrive/Colab Notebooks/NLP/Week 1/Lab 1/valid1.txt'

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [22]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)
        

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

In [ ]:
#build_dict(train1)

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [23]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [24]:
def softmax(x):
    ### FILL CODE
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [25]:
#testing the softmax function
x = np.random.rand(1, 10) + 100
print(softmax(x))
print('\n') 
print(np.sum(softmax(x)))

[[0.06906438 0.05963928 0.14750206 0.14633018 0.0822006  0.14538467
  0.09059219 0.10984707 0.07226521 0.07717435]]


1.0


Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [34]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    lr = 0.09
    for iter in range(niter):
        ### FILL CODE
        loss = 0
        for y,x in data:
          x=x.reshape(-1,1)
          #print(x.shape)
          y_ = np.zeros((nlabels,1))
          #print(y_.shape)
          y_[y] = 1

          #predicted probabilities
          pred = softmax(w @ x)
          
          loss +=-np.log(pred)
          #print(loss)

          #gradient
          grad = -(y_ - pred) @ x.T

          w = w - lr * grad


    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [35]:
from numpy.core.fromnumeric import argmax
def predict(w, x):
    ## FILL CODE
    pred = softmax(w@x)
    return np.argmax(pred) #class with highest probability

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [36]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    accuracy = 0
    for y_i,x_i in valid_data:
      pred = predict(w,x_i)
      #print(pred)
      if pred == y_i:
        accuracy +=1

    acc = accuracy/len(valid_data)
    

    return acc



In [37]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict(train1)
train_data = load_data(train1, word_dict, label_dict)
valid_data = load_data(valid1, word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 0.921



In [ ]:
#len(valid_data)

In [ ]:
#word_dict, label_dict = build_dict(filename)

In [ ]:
#train_data = load_data(filename, word_dict, label_dict)
#word_dict